In [1]:
from dotenv import load_dotenv
import os 

load_dotenv(".env")

URL = os.getenv("URL")
USER_ID = os.getenv("USER_ID")
API_KEY = os.getenv("API_KEY")

In [92]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import requests

columns_to_keep = [
    "Name", "PremiereDate", "CriticRating", "OfficialRating", "Overview", "Taglines",
    "Genres", "CommunityRating", "RunTimeTicks", "ProductionYear", "People", "Studios", "UserData"
]

class JellyfinClient(BaseEstimator, TransformerMixin):
    def __init__(self, url, user_id, api_key, played_status="IsPlayed", limit=None):
        """
        Initializes the Jellyfin Client.
        :param url: Base URL of the Jellyfin server.
        :param user_id: User ID for the API request.
        :param api_key: API key for authentication.
        :param played_status: "IsPlayed" or "IsNotPlayed" to filter movies.
        """
        self.url = url
        self.user_id = user_id
        self.api_key = api_key
        self.played_status = played_status  # Switch for IsPlayed or IsNotPlayed
        self.limit = limit

    def get_amount(self):
        """Fetch the total number of movies with the current filter."""
        res = requests.get(
            f"{self.url}/emby/Users/{self.user_id}/Items"
            f"?StartIndex=0&Limit=1&Recursive=true&IncludeItemTypes=Movie"
            f"&api_key={self.api_key}&Filters={self.played_status}"
        )
        res.raise_for_status()
        return res.json().get("TotalRecordCount")

    def get_chunk(self, start, chunk):
        """Fetch a chunk of movies with the current filter."""
        res = requests.get(
            f"{self.url}/emby/Users/{self.user_id}/Items"
            f"?StartIndex={start}&Limit={chunk}&Recursive=true&IncludeItemTypes=Movie"
            f"&api_key={self.api_key}&Filters={self.played_status}"
            f"&Fields=Budget,Genres,Overview,People,Revenue,Studios,Taglines,ProviderIds,"
            f"CriticRating,OfficialRating,PremiereDate,CommunityRating,RunTimeTicks,ProductionYear,UserData"
        )
        res.raise_for_status()
        return res.json().get("Items")

    def fit(self, X, y=None):
        return self

    def transform(self, X=None):
        """Fetch all movies in chunks based on the current filter."""
        size = self.limit or self.get_amount()

        start = 0
        chunk = 50
        all_movies = []
        while start < size:
            print(
                f"Fetching movies {start} - {start + chunk} of {size} ({self.played_status})")
            all_movies.extend(self.get_chunk(start, chunk))
            start += chunk
        return all_movies


class MovieDataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self, columns_to_keep):
        self.columns_to_keep = columns_to_keep

    def filter_movie(self, movie):
        """Filter and clean a single movie dictionary."""
        filtered = {key: value for key,
                    value in movie.items() if key in self.columns_to_keep}

        # Process nested People and Studios
        if "People" in filtered:
            filtered["People"] = [[person.get("Id"), person.get(
                "Type")] for person in filtered["People"]]
        if "Studios" in filtered:
            filtered["Studios"] = [studio["Id"]
                                   for studio in filtered["Studios"] if "Name" in studio]
        if "UserData" in filtered:
            filtered["IsFavorite"] = filtered["UserData"].get("IsFavorite")
            del filtered["UserData"]

        if "Taglines" in filtered:
            filtered["Taglines"] = ("\n").join(filtered["Taglines"])

        return filtered

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.DataFrame([self.filter_movie(movie) for movie in X])

In [94]:
data_pipeline = Pipeline([
    ("jellyfin_client", JellyfinClient(URL, USER_ID, API_KEY, played_status="IsPlayed", limit=5)),
    ("data_cleaner", MovieDataCleaner(columns_to_keep)),
])

dp = data_pipeline.fit_transform(None)
# dp.columns

Fetching movies 0 - 50 of 5 (IsPlayed)


In [4]:
# dp.columns
# for col in dp.columns: 
#     print(f'Column: {col} has null: {dp[col].isnull().values.any()}')

Column: Name has null: False
Column: PremiereDate has null: False
Column: CriticRating has null: True
Column: OfficialRating has null: True
Column: Overview has null: False
Column: Taglines has null: False
Column: Genres has null: False
Column: CommunityRating has null: False
Column: RunTimeTicks has null: True
Column: ProductionYear has null: False
Column: People has null: False
Column: Studios has null: False
Column: IsFavorite has null: False


In [ ]:
import fasttext
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler, OneHotEncoder
from scipy.sparse import hstack, csr_matrix
from collections import Counter


class MovieFeatureEngineerWithFastText(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.fasttext_name_model = None
        self.fasttext_overview_model = None
        self.fasttext_people_model = None
        self.fasttext_studios_model = None

        # Encoders and scalers
        self.ohe_rating = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
        self.mlb_genres = MultiLabelBinarizer()
        self.scaler = MinMaxScaler()

    def train_fasttext_model(self, text_data, vector_size=100):
        # Save temporary data
        temp_filename = 'temp_fasttext.txt'
        text_data.dropna().to_csv(temp_filename, index=False, header=False)
        # Train the model
        model = fasttext.train_unsupervised(temp_filename, model='skipgram', dim=vector_size)
        return model

    def fit(self, X, y=None):
        # Fit numerical scaler
        X_filled = X[['CriticRating', 'CommunityRating', 'RunTimeTicks', 'ProductionYear']].fillna(-7)
        self.scaler.fit(X_filled)

        # Train FastText models
        self.fasttext_name_model = self.train_fasttext_model(X['Name'].fillna("unknown"))
        self.fasttext_overview_model = self.train_fasttext_model(X['Overview'].fillna("unknown"))
        self.fasttext_people_model = self.train_fasttext_model(
            X['People'].apply(lambda p: ' '.join([item[0] for item in p if isinstance(item, list)])).fillna("unknown")
        )
        self.fasttext_studios_model = self.train_fasttext_model(X['Studios'].fillna("unknown"))

        # Fit encoders for categorical and multilabel features
        self.ohe_rating.fit(X[['OfficialRating']].fillna("unknown"))
        self.mlb_genres.fit(X['Genres'].fillna("unknown"))
        return self

    def transform(self, X):
        df = X.copy()

        # Transform numerical features
        numerical_features = ['CriticRating', 'CommunityRating', 'RunTimeTicks', 'ProductionYear']
        df[numerical_features] = df[numerical_features].fillna(-7)
        scaled_numerical = csr_matrix(self.scaler.transform(df[numerical_features]))

        # Helper function for FastText embeddings
        def get_fasttext_vector(text, model):
            if isinstance(text, str):
                vectors = [model.get_word_vector(word) for word in text.split() if word in model.words]
                if vectors:
                    return np.mean(vectors, axis=0)
            return np.zeros(model.get_dimension())

        # Generate FastText embeddings
        name_embeddings = np.vstack(df['Name'].fillna("unknown").apply(lambda x: get_fasttext_vector(x, self.fasttext_name_model)))
        overview_embeddings = np.vstack(df['Overview'].fillna("unknown").apply(lambda x: get_fasttext_vector(x, self.fasttext_overview_model)))
        people_embeddings = np.vstack(
            df['People'].apply(
                lambda p: get_fasttext_vector(
                    ' '.join([item[0] for item in p if isinstance(item, list)]), self.fasttext_people_model)
            )
        )
        studios_embeddings = np.vstack(df['Studios'].fillna("unknown").apply(lambda x: get_fasttext_vector(x, self.fasttext_studios_model)))

        # Debug: Check dimensions of all embeddings
        print(f"Name embeddings shape: {name_embeddings.shape}")
        print(f"Overview embeddings shape: {overview_embeddings.shape}")
        print(f"People embeddings shape: {people_embeddings.shape}")
        print(f"Studios embeddings shape: {studios_embeddings.shape}")

        # Convert FastText embeddings (dense arrays) to sparse format
        name_embeddings_sparse = csr_matrix(name_embeddings)
        overview_embeddings_sparse = csr_matrix(overview_embeddings)
        people_embeddings_sparse = csr_matrix(people_embeddings)
        studios_embeddings_sparse = csr_matrix(studios_embeddings)

        # Count role occurrences
        def count_roles(people):
            role_counter = Counter()
            for person in people:
                if isinstance(person, list) and len(person) > 1:
                    role_counter[person[1]] += 1
            return role_counter

        role_counts = df['People'].apply(count_roles)
        role_features = csr_matrix(np.array([
            [role_counts.get('Actor', -7),
             role_counts.get('Director', -7),
             role_counts.get('Writer', -7),
             role_counts.get('Producer', -7)]
            for role_counts in role_counts
        ]))

        # Date encoding
        df["PremiereDate"] = pd.to_datetime(df['PremiereDate'])
        df['year'] = df['PremiereDate'].dt.year.astype(int)
        df['month'] = df['PremiereDate'].dt.month.astype(int)
        df['day'] = df['PremiereDate'].dt.day.astype(int)
        df['day_of_week'] = df['PremiereDate'].dt.dayofweek.astype(int)
        df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
        df['quarter'] = df['PremiereDate'].dt.quarter.astype(int)
        df['week_of_year'] = df['PremiereDate'].dt.isocalendar().week.astype(int)
        df['day_of_year'] = df['PremiereDate'].dt.dayofyear.astype(int)
        df = df.drop(columns=['PremiereDate'])
        date_features = csr_matrix(df[['year', 'month', 'day', 'day_of_week', 'is_weekend', 'quarter', 'week_of_year', 'day_of_year']].values)

        # Encode categorical and multilabel features
        official_rating_encoded = self.ohe_rating.transform(df[['OfficialRating']].fillna("unknown"))
        genres_encoded = csr_matrix(self.mlb_genres.transform(df['Genres'].fillna("unknown")))

        # Stack all features together
        final_sparse_matrix = hstack([
            scaled_numerical,               # Scaled numerical data
            name_embeddings_sparse,         # Sparse FastText embeddings for Name
            overview_embeddings_sparse,     # Sparse FastText embeddings for Overview
            people_embeddings_sparse,       # Sparse FastText embeddings for People
            studios_embeddings_sparse,      # Sparse FastText embeddings for Studios
            role_features,                  # Sparse role count features
            date_features,                  # Sparse date encoding features
            official_rating_encoded,        # Sparse one-hot encoded Official Rating
            genres_encoded                  # Sparse multi-label binarized Genres
        ])

        return final_sparse_matrix

In [ ]:
# Full pipeline
full_pipeline = Pipeline([
    ('feature_engineer', MovieFeatureEngineer()),
])

# Split data
df = dp.drop(columns=['IsFavorite'])
y = dp['IsFavorite']

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.2, random_state=42)

# Apply pipeline and resampling
X_train_transformed = full_pipeline.fit_transform(X_train)
X_test_transformed = full_pipeline.transform(X_test)  # Use transform instead of fit_transform for test data


In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, precision_recall_curve, classification_report, confusion_matrix
)
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, recall_score
from sklearn.pipeline import Pipeline

# Full pipeline
full_pipeline = Pipeline([
    ('feature_engineer', MovieFeatureEngineer()),
])

# Split data
df = dp.drop(columns=['IsFavorite'])
y = dp['IsFavorite']

X_train, X_test, y_train, y_test = train_test_split(
    df, y, test_size=0.2, random_state=42)

# Apply pipeline and resampling
X_train_transformed = full_pipeline.fit_transform(X_train)
X_test_transformed = full_pipeline.transform(X_test)  # Use transform instead of fit_transform for test data

In [ ]:
# Hyperparameter grid
param_dist = {
    'n_estimators': [200, 500, 1000, 2000, 4000],          # Number of trees in the forest
    'max_depth': [10, 20, 50, None],           # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],           # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],             # Minimum samples required at each leaf node
    'max_features': ['sqrt', 'log2', None],    # Number of features considered for the best split
    'bootstrap': [True, False],                # Use bootstrap samples
    'criterion': ['gini', 'entropy'],          # Split quality criterion
}

# Model and scoring
rf = RandomForestClassifier(class_weight="balanced", random_state=42)
recall_scorer = make_scorer(recall_score)

# RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    scoring=recall_scorer,
    n_iter=20,
    cv=5,
    verbose=2,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train_transformed, y_train)

In [ ]:
# Best model
tuned_rf = random_search.best_estimator_
print("Best Parameters:", random_search.best_params_)

# Predict probabilities for AUC evaluation
y_pred_proba = tuned_rf.predict_proba(X_test_transformed)[:, 1]
y_pred = tuned_rf.predict(X_test_transformed)

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Print metrics
print("Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
new_pipeline = Pipeline([
    ("jellyfin_client", JellyfinClient(URL, USER_ID, API_KEY, played_status="IsUnPlayed")),
    ("data_cleaner", MovieDataCleaner(columns_to_keep)),
    
])

new_df = new_pipeline.fit_transform(None)
X_new_transformed = full_pipeline.transform(new_df)

In [78]:
predictions = tuned_rf.predict(X_new_transformed)
predicted_probabilities = tuned_rf.predict_proba(X_new_transformed)[:, 1]

results_df = new_df[['Name']].copy()  # or 'MovieID', if you have it
results_df['PredictedIsFavorite'] = predictions
results_df['ProbabilityIsFavorite'] = predicted_probabilities

# results_df
print(("\n").join(results_df[results_df['ProbabilityIsFavorite'] > 0.6]['Name']))

The Edge
Operation Fortune: Ruse de Guerre
Black Hawk Down
Beverly Hills Cop: Axel F
Snowden
Adventureland
Wind River
Silence
The Bikeriders
Love Actually
Bound by Honor
End of Watch
Fallen
House on Haunted Hill
The Infiltrator
Angel Has Fallen
Punch-Drunk Love
Strange Darling
Stillwater
To Die For
LEGO Scooby-Doo! Haunted Hollywood
Public Enemies
Pearl Harbor
Con Air
12 Strong
Hit Man
True Romance
The Dreamers
Unforgiven
The Fly
Natural Born Killers
2 Guns
Arbitrage
The Frighteners
The Crow
The Report
The Strangers: Prey at Night
MaXXXine
Carlito's Way
Rounders
Fifty Shades Freed
Longlegs
LEGO Scooby-Doo! Blowout Beach Bash
The Killing of a Sacred Deer
I Origins
Triangle of Sadness
Cold Mountain
Tombstone
Bound
Bad Boys for Life
Affliction
The House That Jack Built
Resident Evil: Death Island
American Psycho
The Lighthouse
The Big Short
The Firm
Bad Boys: Ride or Die
Legend
Expend4bles
Conspiracy Theory
White Boy Rick
Broken City
The Accountant
The Devil's Advocate
G.I. Jane
